# Applied Data Analysis Project
**Team**: ToeStewBrr - Alexander Sternfeld, Marguerite Thery, Antoine Bonnet, Hugo Bordereaux

**Dataset**: CMU Movie Summary Corpus


## 3. CoreNLP Analysis

We first load data files and download the pre-processed dataframes. 

In [ ]:
from load_data import *
from coreNLP_analysis import *

download_data()
plot_df = load_plot_df()
movie_df = load_movie_df()
char_df = load_char_df()
names_df = load_names_df()
cluster_df = load_cluster_df()

## 3.1. Extracting characters

For any character, we want to extract related information (from name clusters, character metadata) as well as actions, characteristics and relations (from CoreNLP). We first extract information from the pre-processed dataframes. 

We will use Harry Potter's character as an example

In [ ]:
char_name = 'Harry Potter'
movie_ids = list(char_df[char_df['Character name'] == 'Harry Potter']['Wikipedia ID'])
char_ids = names_df.loc[char_name].values[0]
trope = cluster_df.loc[cluster_df['Character name'] == char_name]
# if trop is empty, set trope to None
if trope.empty:
    trope = None

print('Movies with character', char_name, ':')
print('\tMovie IDs:', movie_ids)
print('\tCharacter IDs:', char_ids)
print('\tTrope:', trope)

movie_id = movie_ids[3] 
print('Selecting movie ID as example:', movie_id)

We now extract information from the CoreNLP plot summary analysis. Each xml file has a tree structure, which we can parse as shown below. 

In [ ]:
# To get the tree from xml CoreNLP output
def get_tree(movie_id):
    xml_filename = os.path.join(XML_DIR, '{}.xml'.format(movie_id))
    tree = ET.parse(xml_filename)
    return tree

We can extract all parsed sentences from the xml file, each of which we can view as a tree structure. 

In [ ]:
# Given an xml file, we return all of its CoreNLP parsed sentences 
def get_parsed_sentences(tree):
    sentences = []
    root = tree.getroot()
    for child in tree.iter():
        if child.tag == "parse":
            sentences.append(child.text)
    return sentences

# To print parsed sentences as a pretty tree. 
def print_tree(parsed_string):
    tree = Tree.fromstring(parsed_string)
    tree.pretty_print()


In [ ]:
tree = get_tree(movie_id)
parsed_str = get_parsed_sentences(tree)[5]
print_tree(parsed_str)

We also want to extract all character names from the xml file. Note that we aggregate consecutive words tagged as NNP (noun, proper, singular) as the same character name (this assumes that plot summaries never contain two distinct names side by side without delimiting punctuation). This is a reasonable assumption since list of names are almost always separated by commas. 

In [ ]:
def get_characters(tree):
    characters = []
    current_word = None
    was_person = False
    character = ''
    for child in tree.iter():
        if child.tag == 'word':
            current_word = child.text
        if child.tag == 'NER' and child.text == 'PERSON':
            if was_person:# Continue the character
                character += ' ' + current_word
            else: # Start the character
                character = current_word
                was_person = True
        if was_person and child.tag == 'NER' and child.text != 'PERSON': # End the character
            characters.append(character)
            character = ''
            was_person = False
    return characters

In [ ]:
characters = get_characters(tree)
characters[:15]

In [ ]:
# DISCARD THIS METHOD?

# Given a character in a movie, find all sentences mentioning the character
def sentences_with_character(xml_filename, char_name):
    char_sentences = []
    if os.path.isfile(xml_filename):
        sentences = get_parsed_sentences(xml_filename)
        char_sentences = [sentence for sentence in sentences if char_name in sentence]
    return char_sentences

Notice that some characters are sometimes mentioned by their full name, and sometimes by a partial name (e.g. Harry Potter is most often mentioned as simply Harry). To get a more precise idea of how many times each character is mentioned, we wish to denote each character by their full name, i.e. the longest version of their name that appears in the plot summary. 

To optimize full name lookup, for each plot summary we construct a dictionary which stores as key every partial name mentioned, and as corresponding values the full name of each character.  

In [ ]:
def get_full_name(string, characters):
    ''' 
    Find the longest name of a given character in a list of character names. 
    Input: 
        string: character name (partial or full)
        characters: list of character names
    Output: 
        full_name: longest name of character found in characters
    '''
    names = string.split(' ')
    max_length = 0
    for character in characters:
        char_names = character.split(' ')
        if set(names) <= set(char_names): 
            num_names = len(char_names)
            if num_names > max_length:
                max_length = num_names
                full_name = character
    return full_name

# Helper function: given a list of characters, make a dictionary with all maps (short name : full name)
def full_name_dict(characters): 
    full_names = {}
    for character in characters:
        full_names[character] = get_full_name(character, characters)
    return full_names


In [ ]:
# Toy example
characters_example = ['Harry Potter', 'Harry', 'Lord Voldemort', 'Harry James Potter', 'Dumbledore', 'Albus Dumbledore']
print('Full name:', get_full_name('Harry Potter', characters_example))
print('All full name pairs:', full_name_dict(characters_example))


We can now construct a dictionary with keys being the characters' full name and values being the number of times any version of their name is mentioned. 

In [ ]:
# Given a list of characters names, create a dictionary of characters with 
# keys being their full name and values being the number of times they appear in the list

def aggregate_characters(characters):
    ''' 
    Input: list of characters
    Output: dictionary of (full name : number of times name is mentioned in list)
    Example: ['Harry Potter', 'Voldemort', 'Harry'] -> {'Harry Potter': 2, 'Voldemort': 1}
    '''
    character_dict = dict()
    for character in characters:
        full_character = get_full_name(character, characters)
        if full_character in character_dict:
            character_dict[full_character] += 1
        else:
            character_dict[full_character] = 1
    return character_dict


In [ ]:
aggregate_characters(characters)

We can now extract the most mentioned characters in any plot summary. 

In [ ]:
# Given a parse tree, extract the most mentioned characters in decreasing order
def most_mentioned(tree):
    '''
    Input: 
        tree: parse tree of the xml file
        N: the number of characters to return
    Output:
        A dictionary of the N characters most mentioned in the movie
    '''
    characters = get_characters(tree)
    character_dict = aggregate_characters(characters)
    sorted_characters = sorted(character_dict.items(), key=lambda x: x[1], reverse=True)
    return sorted_characters

In [ ]:
most_mentioned(tree)[:10]

 ### 3.2. Extracting relationships

 We cannot extract character interactions directly from the CoreNLP output (or can we?). Instead, we use the number of common mentions of two characters in the same sentence as a proxy for the number of interactions. We first define a method that gets the full name of all characters mentioned in each sentence. 

In [ ]:
# Given a list of characters and a string sentence, find the sequence of characters appearing in the sentence
def characters_in_sentence(characters, sentence):
    '''
    Input: 
        characters: list of characters
        sentence: string
    Output: 
        characters_mentioned: list of characters that appear in the sentence
        characters: list of remaining characters
    Example: characters_in_sentence(['Harry Potter', 'Voldemort', 'Dumbledore'], 'Harry Potter fights Voldemort bravely.')
    Output: ['Harry Potter', 'Voldemort'], ['Dumbledore']
    '''
    characters_mentioned = []
    if (len(characters) == 0):
            return characters_mentioned, characters
    character = characters.pop(0)
    while character in sentence:
        sentence = sentence.split(character, 1)[1]
        characters_mentioned.append(character)
        if (len(characters) == 0):
            break
        character = characters.pop(0)
    return characters_mentioned, characters



In [ ]:
plot_summary = plot_df.loc[plot_df['Wikipedia ID'] == movie_id]['Summary'].values[0]
sentences = re.split(r'(?<=[.!?])\s+', plot_summary)
print('First sentence:', sentences[0])
print('Characters:', characters[:10])
characters_mentioned, characters_rem = characters_in_sentence(characters[:10], sentences[0])
print('Characters mentioned:', characters_mentioned)
print('Remaining characters:', characters_rem)

We define a method that takes in a movie ID, and outputs the number of common mentions (i.e. interactions) for each pair of characters. 

In [ ]:

def character_pairs(movie_id):
    ''' 
    Find all pairs of characters that appear in the same sentence in a movie plot summary. 
    Input: 
        movie_id: integer Movie ID
    Output:
        A list of all character pairs in the movie in decreasing order of frequency
    '''
    char_pairs = dict()

    # Parse xml file and get all characters from plot summary
    xml_filename = os.path.join(XML_DIR, '{}.xml'.format(movie_id))
    tree = ET.parse(xml_filename)
    characters = get_characters(tree)
    full_name_map = full_name_dict(characters) # all maps (partial name : full name)

    # Split plot summary into sentences
    plot_summary = plot_df.loc[plot_df['Wikipedia ID'] == movie_id]['Summary'].values[0]
    sentences = re.split(r'(?<=[.!?])\s+', plot_summary)

    # For each sentence, find the characters mentioned and their full names, then add count for each pair between them
    for sentence in sentences:
        # Only consider sentences with at least 2 mentioned characters
        characters_mentioned, characters = characters_in_sentence(characters, sentence)
        if len(characters_mentioned) >= 2: 
            
            # Get full names of each character mentioned, remove doubles
            full_mentioned = set([full_name_map[c] for c in characters_mentioned])

            # Add count for each pair of characters
            pairs = list(itertools.combinations(sorted(full_mentioned), 2))
            for pair in pairs:
                if pair in char_pairs:
                    char_pairs[pair] += 1
                else:
                    char_pairs[pair] = 1
    
    # Sort character pairs by number of times they appear together
    sorted_pairs = sorted(char_pairs.items(), key=lambda x: x[1], reverse=True)
    return sorted_pairs

In [ ]:
character_pairs(movie_id)[:10]

In [ ]:
# Given a movie ID, find the character pairs with the highest number of interactions (both mentioned in a single sentence).
def most_interactions(movie_id):
    # Get all characters in the movie
    xml_filename = os.path.join(XML_DIR, '{}.xml'.format(movie_id))
    tree = ET.parse(xml_filename)
    characters = get_characters(tree)

    # Find the plot summary sentences with at least two characters 
    plot_summary = plot_df.loc[plot_df['Wikipedia ID'] == movie_id]['Summary'].values[0]
    sentences = re.split(r'(?<=[.!?])\s+', plot_summary)
    
    # Find the character pairs with the most interactions
    character_pairs = dict()
    for sentence in sentences:
        characters = get_characters(sentence)
        character_pairs = aggregate_characters(characters)
    sorted_pairs = sorted(character_pairs.items(), key=lambda x: x[1], reverse=True)
    return sorted_pairs


In [ ]:
xml_filename = os.path.join(XML_DIR, '{}.xml'.format(movie_id))
tree = ET.parse(xml_filename)
    

In [ ]:
plot_summary = plot_df.loc[plot_df['Wikipedia ID'] == movie_id]['Summary'].values[0]
sentences = re.split(r'(?<=[.!?])\s+', plot_summary)
    

In [ ]:
most_interactions(movie_id)[:10]

We would like to identify the two main characters in a plot summary. 

In [ ]:
#TODO: only consider romantic movies
#TODO: Aggregate consecutive entity names inton one
extracted_dir = 'Data/CoreNLP/corenlp_plot_summaries_xml'

def make_pairs (extracted_dir): 
  pairs = []
  for filename in os.listdir(extracted_dir):
      f = os.path.join(extracted_dir, filename) 
      if os.path.isfile(f):
          # Create characters list for each file 
          characters = []
          tree = ET.parse(f)
          root = tree.getroot()
          for child in tree.iter():
              if child.tag == "word":
                current_word = child.text
              if child.tag == "NER": 
                if child.text == "PERSON":
                  characters.append(current_word)
          # Select the two characters which appear most often in the file
          values, counts = np.unique(characters, return_counts=True)
          two_most_frequent_characters = values[counts.argsort()[-2:][::-1]]
          # If there exists two characters, create a pair 
          if len(two_most_frequent_characters) > 1:
            pairs.append([f.replace('Data/CoreNLP/corenlp_plot_summaries_xml/', '').replace('.xml', ''), two_most_frequent_characters[0], two_most_frequent_characters[1]])
  return pairs
                           

In [ ]:
# Convert into an array and as a dataframe
pairs = make_pairs(XML_DIR)
pairs = np.asarray(pairs).reshape(-1, 3)  
pairs_df = pd.DataFrame(pairs, columns=['Wikipedia ID', 'char1', 'char2']) 
pairs_df        

In [ ]:
# Merge pairs dataset with characters 
char_df['Wikipedia ID'] = char_df['Wikipedia ID'].astype(str)
pairs_df['Wikipedia ID'] = pairs_df['Wikipedia ID'].astype(str)
pairs_char = pairs_df.merge(char_df, on="Wikipedia ID")

# Filter out the nan values
pairs_char = pairs_char[~pairs_char['Character name'].isna()]

# Create columns which indicates if char1 and char2 are in character name 
pairs_char['char1_is_in_name'] = pairs_char.apply(lambda x: 1 if x['char1'] in x['Character name'] else 0, axis=1)
pairs_char['char2_is_in_name'] = pairs_char.apply(lambda x: 1 if x['char2'] in x['Character name'] else 0, axis=1)
pairs_char[pairs_char['char1_is_in_name'] == 1 & pairs_char['char2_is_in_name'] == 1]

### 3.3. CoreNLP Analysis

To use the powerful CoreNLP model, first [download it](https://stanfordnlp.github.io/CoreNLP/download.html), then cd into the downloaded `stanford-corenlp` directory. If you have Java, you can run the following command to open a CoreNLP shell: 


`java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer` 
 
Now that the shell is running, we can use their models to annotate some sentences. 

In [ ]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')
nlp.annotate('Barack Obama was born in Hawaii.  He is the president. Obama was elected in 2008.')

